In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#✅ 데이터셋 불러오기

In [ ]:
notop_one_df = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/datathon/주력_비주력상품 데이터/비주력상품/noTop3_one_time_buyers_all_events.parquet') # 비주력상품 1회 구매고객 데이터

In [ ]:
notop_multi_df = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/datathon/주력_비주력상품 데이터/비주력상품/noTop3_multi_buyers_all_events.parquet') # 비주력상품 2회 이상 구매고객 데이터

In [ ]:
import pandas as pd

def _ensure_event_datetime(df, date_col='event_date', hour_col='event_hour', dt_col='event_datetime'):
    """
    event_datetime이 없으면 date+hour로 생성.
    date_col: object/string 날짜
    hour_col: int(hour)
    """
    if dt_col in df.columns:
        return df
    out = df.copy()
    out[date_col] = pd.to_datetime(out[date_col], errors='coerce')
    out[dt_col] = out[date_col] + pd.to_timedelta(out[hour_col], unit='h')
    return out

def calc_disjoint_view_purchase_rates(
    df,
    user_col='user_id',
    type_col='event_type',
    dt_col='event_datetime'
):
    """
    분모: view가 있었던 고유 유저 수
    분자1(view_cart_purchase): view < cart < purchase 를 만족하는 유저(최초 view 이후 최초 purchase 사이에 cart가 존재)
    분자2(view_purchase_only): view < purchase 이면서 그 사이에 cart가 전혀 없는 유저
    ※ 두 분자는 서로 겹치지 않도록 엄격 분리. 동시간(같은 hour) 타이는 제외(<, > 사용).
    반환: 각 분자/분모/전환율과 유저 id 집합(원하면 제거 가능)
    """
    # 준비
    df = _ensure_event_datetime(df, dt_col=dt_col).copy()
    df = df.dropna(subset=[dt_col, user_col, type_col])
    # 한 번 정렬(안정 정렬로 재현성 확보; 동시간 타이는 이후 비교식에서 배제)
    df = df.sort_values([user_col, dt_col, type_col], kind='mergesort')

    # 분모: view 있었던 유저들
    view_users = set(df.loc[df[type_col] == 'view', user_col].unique())
    denom = len(view_users)

    vcp_users = set()      # view→cart→purchase
    vp_only_users = set()  # view→purchase (중간 cart 없음)

    # 유저별 타임라인 스캔
    for uid, g in df.groupby(user_col):
        if uid not in view_users:
            continue

        # 이 유저의 이벤트 시퀀스 (시간 오름차순)
        ev = g[[dt_col, type_col]].to_numpy()

        # 1) 첫 view 시각
        t_view = None
        for t, et in ev:
            if et == 'view':
                t_view = t
                break
        if t_view is None:
            continue

        # 2) 첫 view 이후의 '최초 purchase' 시각 (동시각은 제외: t > t_view)
        t_purchase = None
        for t, et in ev:
            if t > t_view and et == 'purchase':
                t_purchase = t
                break
        if t_purchase is None:
            # view는 있었지만 purchase가 없으면 분자 둘 다 미포함
            continue

        # 3) view와 그 '최초 purchase' 사이에 cart가 하나라도 있는지?
        has_cart_between = False
        for t, et in ev:
            if (t > t_view) and (t < t_purchase) and (et == 'cart'):
                has_cart_between = True
                break

        if has_cart_between:
            vcp_users.add(uid)
        else:
            vp_only_users.add(uid)

    # 두 집합이 겹치지 않도록 검증 (설계상 0이어야 함)
    overlap = vcp_users & vp_only_users
    assert len(overlap) == 0, f"분자 집합이 겹칩니다: {len(overlap)}명"

    vcp_num = len(vcp_users)
    vp_only_num = len(vp_only_users)

    rates = {
        'view_cart_purchase': {
            'numerator_users': vcp_users,
            'numerator': vcp_num,
            'denominator': denom,
            'not_converted': max(denom - vcp_num, 0),
            'rate': (vcp_num / denom) if denom else 0.0
        },
        'view_purchase_only': {
            'numerator_users': vp_only_users,
            'numerator': vp_only_num,
            'denominator': denom,
            'not_converted': max(denom - vp_only_num, 0),
            'rate': (vp_only_num / denom) if denom else 0.0
        },
        'denominator_view_users': denom,
        'overlap_check': len(overlap)  # 항상 0이어야 함
    }
    return rates


In [ ]:
# notop_one_df = 비주력 1회 구매자 데이터셋
# notop_multi_df = 비주력 2회 이상 구매자 데이터셋

res_once = calc_disjoint_view_purchase_rates(notop_one_df)
res_multi = calc_disjoint_view_purchase_rates(notop_multi_df)

def pretty_print(title, res):
    print(f"\n[{title}] 분모(view 유저 수) = {res['denominator_view_users']:,}")
    for k in ['view_cart_purchase', 'view_purchase_only']:
        v = res[k]
        print(f"- {k}: 분자={v['numerator']:,} / 분모={v['denominator']:,} "
              f"(미전환={v['not_converted']:,}) → 전환율={v['rate']:.2%}")
    print(f"분자 집합 겹침 수(overlap_check) = {res['overlap_check']} (항상 0이어야 함)")

pretty_print("비주력 1회 구매자", res_once)
pretty_print("비주력 2회 이상 구매자", res_multi)


## 📈 비주력상품 EDA

### 💵 1번만 구매

In [ ]:
import pandas as pd

def _ensure_event_datetime(df, date_col='event_date', hour_col='event_hour', dt_col='event_datetime'):
    """
    event_datetime이 없으면 date+hour로 생성.
    date_col: object/string 날짜
    hour_col: int(hour)
    """
    if dt_col in df.columns:
        return df
    out = df.copy()
    out[date_col] = pd.to_datetime(out[date_col], errors='coerce')
    out[dt_col] = out[date_col] + pd.to_timedelta(out[hour_col], unit='h')
    return out

def calc_disjoint_view_purchase_rates(
    df,
    user_col='user_id',
    type_col='event_type',
    dt_col='event_datetime'
):
    """
    분모: view가 있었던 고유 유저 수
    분자1(view_cart_purchase): view < cart < purchase 를 만족하는 유저(최초 view 이후 최초 purchase 사이에 cart가 존재)
    분자2(view_purchase_only): view < purchase 이면서 그 사이에 cart가 전혀 없는 유저
    ※ 두 분자는 서로 겹치지 않도록 엄격 분리. 동시간(같은 hour) 타이는 제외(<, > 사용).
    반환: 각 분자/분모/전환율과 유저 id 집합(원하면 제거 가능)
    """
    # 준비
    df = _ensure_event_datetime(df, dt_col=dt_col).copy()
    df = df.dropna(subset=[dt_col, user_col, type_col])
    # 한 번 정렬(안정 정렬로 재현성 확보; 동시간 타이는 이후 비교식에서 배제)
    df = df.sort_values([user_col, dt_col, type_col], kind='mergesort')

    # 분모: view 있었던 유저들
    view_users = set(df.loc[df[type_col] == 'view', user_col].unique())
    denom = len(view_users)

    vcp_users = set()      # view→cart→purchase
    vp_only_users = set()  # view→purchase (중간 cart 없음)

    # 유저별 타임라인 스캔
    for uid, g in df.groupby(user_col):
        if uid not in view_users:
            continue

        # 이 유저의 이벤트 시퀀스 (시간 오름차순)
        ev = g[[dt_col, type_col]].to_numpy()

        # 1) 첫 view 시각
        t_view = None
        for t, et in ev:
            if et == 'view':
                t_view = t
                break
        if t_view is None:
            continue

        # 2) 첫 view 이후의 '최초 purchase' 시각 (동시각은 제외: t > t_view)
        t_purchase = None
        for t, et in ev:
            if t > t_view and et == 'purchase':
                t_purchase = t
                break
        if t_purchase is None:
            # view는 있었지만 purchase가 없으면 분자 둘 다 미포함
            continue

        # 3) view와 그 '최초 purchase' 사이에 cart가 하나라도 있는지?
        has_cart_between = False
        for t, et in ev:
            if (t > t_view) and (t < t_purchase) and (et == 'cart'):
                has_cart_between = True
                break

        if has_cart_between:
            vcp_users.add(uid)
        else:
            vp_only_users.add(uid)

    # 두 집합이 겹치지 않도록 검증 (설계상 0이어야 함)
    overlap = vcp_users & vp_only_users
    assert len(overlap) == 0, f"분자 집합이 겹칩니다: {len(overlap)}명"

    vcp_num = len(vcp_users)
    vp_only_num = len(vp_only_users)

    rates = {
        'view_cart_purchase': {
            'numerator_users': vcp_users,
            'numerator': vcp_num,
            'denominator': denom,
            'not_converted': max(denom - vcp_num, 0),
            'rate': (vcp_num / denom) if denom else 0.0
        },
        'view_purchase_only': {
            'numerator_users': vp_only_users,
            'numerator': vp_only_num,
            'denominator': denom,
            'not_converted': max(denom - vp_only_num, 0),
            'rate': (vp_only_num / denom) if denom else 0.0
        },
        'denominator_view_users': denom,
        'overlap_check': len(overlap)  # 항상 0이어야 함
    }
    return rates


In [ ]:
# notop_one_df = 비주력 1회 구매자 데이터셋
# notop_multi_df = 비주력 2회 이상 구매자 데이터셋

res_once = calc_disjoint_view_purchase_rates(notop_one_df)
res_multi = calc_disjoint_view_purchase_rates(notop_multi_df)

def pretty_print(title, res):
    print(f"\n[{title}] 분모(view 유저 수) = {res['denominator_view_users']:,}")
    for k in ['view_cart_purchase', 'view_purchase_only']:
        v = res[k]
        print(f"- {k}: 분자={v['numerator']:,} / 분모={v['denominator']:,} "
              f"(미전환={v['not_converted']:,}) → 전환율={v['rate']:.2%}")
    print(f"분자 집합 겹침 수(overlap_check) = {res['overlap_check']} (항상 0이어야 함)")

pretty_print("비주력 1회 구매자", res_once)
pretty_print("비주력 2회 이상 구매자", res_multi)



[비주력 1회 구매자] 분모(view 유저 수) = 295,577
- view_cart_purchase: 분자=67,076 / 분모=295,577 (미전환=228,501) → 전환율=22.69%
- view_purchase_only: 분자=144,784 / 분모=295,577 (미전환=150,793) → 전환율=48.98%
분자 집합 겹침 수(overlap_check) = 0 (항상 0이어야 함)

[비주력 2회 이상 구매자] 분모(view 유저 수) = 205,370
- view_cart_purchase: 분자=54,412 / 분모=205,370 (미전환=150,958) → 전환율=26.49%
- view_purchase_only: 분자=138,932 / 분모=205,370 (미전환=66,438) → 전환율=67.65%
분자 집합 겹침 수(overlap_check) = 0 (항상 0이어야 함)


In [ ]:
# notop_one_df 에서 같은 시간에 찍힌 유저의 event_type 확인
def check_same_hour_events(df, user_col='user_id', type_col='event_type', dt_col='event_hour'):
    """
    Checks for users with multiple event types at the same event_datetime.

    Args:
        df: DataFrame to check.
        user_col: Column name for user ID.
        type_col: Column name for event type.
        dt_col: Column name for event datetime.

    Returns:
        A dictionary where keys are user_ids and values are dictionaries
        containing the event_datetime and a list of event_types at that time,
        for users with more than one event type at the same time.
    """
    # Group by user and datetime, then get unique event types and their count
    grouped = df.groupby([user_col, dt_col])[type_col].agg(['unique', 'nunique'])

    # Filter for groups with more than one unique event type
    same_hour_events = grouped[grouped['nunique'] > 1]

    # Format the output
    result = {}
    for (user_id, event_hour), row in same_hour_events.iterrows():
        result[user_id] = {
            dt_col: event_hour,
            type_col: row['unique'].tolist()
        }
    return result

same_hour_violations_one = check_same_hour_events(notop_one_df)

print(f"Number of users in notop_one_df with multiple event types at the same time: {len(same_hour_violations_one)}")

# Display a few examples
if same_hour_violations_one:
    print("\nExamples of users with same-hour events:")
    for user_id, details in list(same_hour_violations_one.items())[:5]:
        print(f"User ID: {user_id}, Datetime: {details['event_datetime']}, Events: {details['event_type']}")

Number of users in notop_one_df with multiple event types at the same time: 293685

Examples of users with same-hour events:


KeyError: 'event_datetime'